# Hexapod currents and Rotator torques

The purpose of this notebook is to compare the hexapod forces (as measured by the strut currents) and the rotator torques between the LSSTCam and the ComCam.  LSSTCam has additional vacuum insulated pipes (VIP) that could cause additional torques.

Craig Lage - 03-Mar-25 


In [ ]:
import sys, time, os, asyncio, glob
import shlex, subprocess
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
client = EfdClient('usdf_efd')

## Set up the necessary subroutines

In [ ]:
def plotHexapod(df_position, df_current, fig, smooth_currents=False):
    hexes = ['X', 'Y', 'Z', 'U', 'V', 'W']
    axs = []
    plotCounter = 1
    for i in range(3):
        axs.append([])
        for j in range(4):
            axs[i].append(fig.add_subplot(3,4,plotCounter))
            plotCounter += 1
    plt.subplots_adjust(wspace=0.5, hspace=1.5)
    for i in range(3):
        for j in range(4):
            if j < 2:
                counter = 3 * j + i
                name = f"position{counter}"
                axs[i][j].set_title(f"Position {hexes[counter]}")
                df_position[name].plot(ax=axs[i][j], color='blue')
            else:
                counter = 3 * (j - 2) + i
                name = f"motorCurrent{counter}"
                axs[i][j].set_title(f"Current {counter}")
                if smooth_currents:
                    test = df_current[name].rolling(10).sum() / 10.0
                    test.plot(ax=axs[i][j], color='green')
                else:
                    df_current[name].plot(ax=axs[i][j], color='green')
    return

def plotRotator(df_position, df_current, fig, smooth_torques=False):
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    ax1.set_title("Rotator Position (degrees)")
    df_position['actualPosition'].plot(ax=ax1, color='blue')
    ax2.set_title("Rotator Torques (Nm)")
    if smooth_torques:
        test0 = df_current['torque0'].rolling(10).sum() / 10.0
        test1 = df_current['torque1'].rolling(10).sum() / 10.0
        test0.plot(ax=ax2, color='green')
        test1.plot(ax=ax2, color='orange')
    else:
        df_current['torque0'].plot(ax=ax2, color='green')
        df_current['torque1'].plot(ax=ax2, color='orange')
    return

## First the hexapod positions and strut currents

In [ ]:
# LSSTCam Hexapod testing with no rotations
start = Time("2025-02-26T15:00:00Z", scale='utc')
end = Time("2025-02-26T19:00:00Z", scale='utc')

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)
print(len(camhex), len(camhex_currents))

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig)
plt.suptitle("LSSTCam Testing 2025-02-26", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_2025-02-26.png')

In [ ]:
# LSSTCam Hexapod testing with maximal rotations
start = Time("2025-02-27T13:00:00Z", scale='utc')
end = Time("2025-02-27T18:00:00Z", scale='utc')

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)
print(len(camhex), len(camhex_currents))

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig)
plt.suptitle("LSSTCam Testing 2025-02-27", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_2025-02-27.png')

In [ ]:
# Typical ComCam observing night.
start = Time("2024-12-12T05:00:00Z", scale='utc')
end = Time("2024-12-12T07:00:00Z", scale='utc')

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)
print(len(camhex), len(camhex_currents))

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig)
plt.suptitle("ComCam observing 2024-12-11", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/ComCam_2024-12-11.png')

In [ ]:
# Blowup of LSSTCam Hexapod testing with no rotations
start = Time("2025-02-26T17:37:00Z", scale='utc')
end = Time("2025-02-26T17:39:00Z", scale='utc')

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)
print(len(camhex), len(camhex_currents))

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig)
plt.suptitle("LSSTCam Testing 2025-02-26", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_Blowup_2025-02-26.png')

## Next the rotator positions and torques

In [ ]:
# Typical ComCam observing night.
start = Time("2024-12-12T05:00:00Z", scale='utc')
end = Time("2024-12-12T07:00:00Z", scale='utc')

rot = await client.select_time_series('lsst.sal.MTRotator.rotation', ['*'], start, end)
mot = await client.select_time_series('lsst.sal.MTRotator.motors', ['*'], start, end)

fig = plt.figure(figsize=(10, 5))
plotRotator(rot, mot, fig)
plt.suptitle("ComCam observing 2024-12-11", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/ComCam_Rotator_2024-12-11.png')

In [ ]:
# Typical ComCam observing night.
start = Time("2024-12-08T05:00:00Z", scale='utc')
end = Time("2024-12-08T10:00:00Z", scale='utc')

rot = await client.select_time_series('lsst.sal.MTRotator.rotation', ['*'], start, end)
mot = await client.select_time_series('lsst.sal.MTRotator.motors', ['*'], start, end)

fig = plt.figure(figsize=(10, 5))
plotRotator(rot, mot, fig)
plt.suptitle("ComCam observing 2024-12-07", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/ComCam_Rotator_2024-12-07.png')

In [ ]:
# LSSTCam Hexapod testing with maximal rotations
start = Time("2025-02-27T13:00:00Z", scale='utc')
end = Time("2025-02-27T18:00:00Z", scale='utc')

rot = await client.select_time_series('lsst.sal.MTRotator.rotation', ['*'], start, end)
mot = await client.select_time_series('lsst.sal.MTRotator.motors', ['*'], start, end)

fig = plt.figure(figsize=(10, 5))
plotRotator(rot, mot, fig)
plt.suptitle("LSSTCam Testing 2025-02-27", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_2025-02-27.png')

## Now make a pdf file with multiple time stamps

In [ ]:
starts = ['2024-12-08T05:00:00Z', '2024-12-12T05:00:00Z', 
          '2025-02-26T15:00:00Z', '2025-02-27T13:00:00Z', 
          '2025-02-28T00:02:00Z', '2025-02-28T17:38:00Z', 
          '2025-02-28T17:57:00Z', '2025-03-01T13:35:00Z']
          
ends =   ['2024-12-08T10:00:00Z', '2024-12-12T07:00:00Z', 
          '2025-02-26T19:00:00Z', '2025-02-27T18:00:00Z', 
          '2025-02-28T01:39:00Z', '2025-02-28T17:49:00Z', 
          '2025-02-28T18:03:00Z', '2025-03-01T15:55:00Z']
labels = ['ComCam', 'ComCam', 'LSSTCam_Test_1', 
          'LSSTCam_Test_2', 'LSSTCam_Test_3', 'LSSTCam_Test_4', 
          'LSSTCam_Test_5', 'LSSTCam_Test_6']

pdf = PdfPages("/home/c/cslage/u/Hexapods/torques/SITCOM-1883_04Mar25.pdf")
fig = plt.figure(figsize=(10, 5))

for i in range(len(starts)):
    date = starts[i].split('T')[0]
    start = Time(starts[i], scale='utc')
    end = Time(ends[i], scale='utc')
    camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
    camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)
    print('Hexapods', labels[i], date, len(camhex), len(camhex_currents))
    plotHexapod(camhex, camhex_currents, fig)
    title = 'Hexapods' + '_' + labels[i] + '_' + date
    plt.suptitle(title, fontsize = 18)
    pdf.savefig(fig)
    plt.clf()
    rot = await client.select_time_series('lsst.sal.MTRotator.rotation', ['*'], start, end)
    mot = await client.select_time_series('lsst.sal.MTRotator.motors', ['*'], start, end)
    print('Rotator', labels[i], date, len(rot), len(mot))
    plotRotator(rot, mot, fig)
    title = 'Rotator' + '_' + labels[i] + '_' + date
    plt.suptitle(title, fontsize = 18)
    pdf.savefig(fig)
    plt.clf()
pdf.close()
print(f"Plots completed and PDF closed")

## Below we are comparing hexapod warm-up ramps.

In [ ]:
# Hexapod warm-up - LSSTCam
#start = Time("2025-02-27T22:05:00Z", scale='utc')
#end = Time("2025-02-27T22:25:00Z", scale='utc')
start = Time("2025-02-27T22:09:10Z", scale='utc')
end = Time("2025-02-27T22:09:14Z", scale='utc')

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig, smooth_currents=True)

plt.suptitle("LSSTCam Hexapod Warm-up 2025-02-27", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_Hexapod_Warm-up_2025-02-27.png')

In [ ]:
# Hexapod warm-up - ComCam
#start = Time("2024-12-06T18:04:00Z", scale='utc')
#end = Time("2024-12-06T18:17:00Z", scale='utc')
start = Time("2024-12-06T18:06:07Z", scale='utc')
end = Time("2024-12-06T18:06:11Z", scale='utc')


camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
camhex_currents = await client.select_time_series('lsst.sal.MTHexapod.electrical', ['*'], start, end, index=1)

fig = plt.figure(figsize=(10, 5))
plotHexapod(camhex, camhex_currents, fig, smooth_currents=True)

plt.suptitle("ComCam Hexapod Warm-up 2024-12-06", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/ComCam_Hexapod_Warm-up_2024-12-06.png')

## Look at rotator torque asymmetry

In [ ]:
# LSSTCam Hexapod testing with maximal rotations
start = Time("2025-02-27T15:04:00Z", scale='utc')
end = Time("2025-02-27T15:08:00Z", scale='utc')

rot = await client.select_time_series('lsst.sal.MTRotator.rotation', ['*'], start, end)
mot = await client.select_time_series('lsst.sal.MTRotator.motors', ['*'], start, end)

fig = plt.figure(figsize=(10, 5))
plotRotator(rot, mot, fig, smooth_torques=True)
plt.suptitle("LSSTCam Testing 2025-02-27", fontsize = 18)
plt.savefig(f'/home/c/cslage/u/Hexapods/torques/LSSTCam_Torque_Asym_2025-02-27.png')